In [ ]:
import torch
import matplotlib.pyplot as plt
import non_local_boxes
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

# Sugar coating for reloading
%matplotlib inline
%load_ext autoreload
%autoreload 2

# in ordert to have unblurred pictures
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

In [ ]:
non_local_boxes.evaluate.nb_columns = 1

---
# I. Test a Wiring $\mathsf{W}$ in a Triangle

In [ ]:
PR = non_local_boxes.utils.PR
PL = non_local_boxes.utils.P_L
PNL = non_local_boxes.utils.P_NL

W_BS09 = torch.tensor([0., 0., 1., 1.,              # f_1(x, a_2) = x
            0., 0., 1., 1.,              # g_1(y, b_2) = y
            0., 0., 0., 1.,              # f_2(x, a_1) = a_1*x
            0., 0., 0., 1.,              # g_2(y, b_1) = b_1*y
            0., 1., 1., 0., 0., 1., 1., 0.,  # f_3(x, a_1, a_2) = a_1 + a_2 mod 2
            0., 1., 1., 0., 0., 1., 1., 0.   # g_3(y, b_1, b_2) = b_1 + b_2 mod 2
            ])

We will draw in blue the collapsing boxes, and in red the ones for which the algorithm does not conclude positively.  First, we evaluate the proportion of blue points in a segment of the triangle to determine if it is interesting to continue drawing the whole triangle.

In [ ]:
matrix_to_tensor = non_local_boxes.utils.matrix_to_tensor
tensor_to_matrix = non_local_boxes.utils.tensor_to_matrix
R = non_local_boxes.evaluate.R
def hflat(P): return float(non_local_boxes.evaluate.h_flat(P))

def proportion_blue_points_in_a_segment(W, Box1, Box3, box_grid_size, max_box_power, threshold):
    
    number_blue_points = 0

    for i in range(box_grid_size+1):
        alpha = i/box_grid_size
        P = alpha*Box1 + (1-alpha)*Box3   # P is a 4x4 matrix
        color_point = "orangered"

        Qright=matrix_to_tensor(torch.clone(P))  # Q is a 2x2x2x2 tensor
        Qcenter=matrix_to_tensor(torch.clone(P))  # Q is a 2x2x2x2 tensor
        Qleft=matrix_to_tensor(torch.clone(P))  # Q is a 2x2x2x2 tensor
        value = max(hflat(Qright), hflat(Qcenter), hflat(Qleft))
        if value > threshold:
            color_point = (0, 0.1, 0.1)
            number_blue_points += 1
        else:
            if value != 0.75:
                for l in range(max_box_power):
                    if color_point != "orangered": break
                    Qright=R(W, tensor_to_matrix(Qright), P)[:,:,:,:,0]
                    Qcenter=R(W, tensor_to_matrix(Qcenter), tensor_to_matrix(Qcenter))[:,:,:,:,0]
                    Qleft=R(W, P, tensor_to_matrix(Qleft))[:,:,:,:,0]
                    value = max(hflat(Qright), hflat(Qcenter), hflat(Qleft))
                    if value > threshold:
                        color_point = (0, 0.1*(1-l/max_box_power)+1*(l/max_box_power), 0.1*(1-l/max_box_power)+1*(l/max_box_power))
                        number_blue_points += 1

    return number_blue_points/box_grid_size

In [ ]:
def test_wiring_triangle(W, Box1, Box1_name, Box2, Box2_name, Box3, Box3_name, box_grid_size, max_box_power, threshold, Details_Are_Printed, good_proportion=0.39):
    # W is a vector of size 32
    n = non_local_boxes.evaluate.nb_columns
    W = torch.t(W.repeat(n, 1))
    # Now W is a 32xn tensor

    # Check the proportion of blue points in the top segment -> stop if there is no enough blue points.
    if proportion_blue_points_in_a_segment(W, Box1, Box3, box_grid_size, 21, threshold)<good_proportion:
        if Details_Are_Printed: print("-- Not collapsing:  ", Box1_name," -- ", Box3_name)
        return ""

    X = torch.tensor([[1,3], [4,1], [4, 5]]).numpy()
    triangle = plt.Polygon(X[:3,:], color="snow")
    plt.gca().add_patch(triangle)
    
    for i in range(box_grid_size+1):
        for j in range(box_grid_size-i+1):
            alpha, beta = i/box_grid_size, j/box_grid_size
            P = alpha*Box1 + beta*Box2 + (1-alpha-beta)*Box3   # P is a 4x4 matrix
            color_point = "orangered"

            Qright=matrix_to_tensor(torch.clone(P))  # Q is a 2x2x2x2 tensor
            Qcenter=matrix_to_tensor(torch.clone(P))  # Q is a 2x2x2x2 tensor
            Qleft=matrix_to_tensor(torch.clone(P))  # Q is a 2x2x2x2 tensor
            value = max(hflat(Qright), hflat(Qcenter), hflat(Qleft))
            if value > threshold:  color_point = (0, 0.1, 0.1)
            else:
                for l in range(max_box_power):
                    if color_point != "orangered":  break
                    Qright=R(W, tensor_to_matrix(Qright), P)[:,:,:,:,0]
                    Qcenter=R(W, tensor_to_matrix(Qcenter), tensor_to_matrix(Qcenter))[:,:,:,:,0]
                    Qleft=R(W, P, tensor_to_matrix(Qleft))[:,:,:,:,0]
                    value = max(hflat(Qright), hflat(Qcenter), hflat(Qleft))
                    if value > threshold:  color_point = (0, 0.1*(1-l/max_box_power)+1*(l/max_box_power), 0.1*(1-l/max_box_power)+1*(l/max_box_power))

            plt.plot(X[0,0]*alpha + X[1,0]*beta + X[2,0]*(1-alpha-beta), X[0,1]*alpha + X[1,1]*beta + X[2,1]*(1-alpha-beta), 'o', markersize=3, color=color_point)
                    
    plt.text(X[0,0], X[0,1]+0.1, Box1_name, horizontalalignment='center')
    plt.text(X[1,0]+0.1, X[1,1], Box2_name, verticalalignment='center')
    plt.text(X[2,0]+0.1, X[2,1], Box3_name, verticalalignment='center')
    handles = []
    handles.append( mlines.Line2D([], [], marker='o',ls='', color=(0, 0.7, 0.7), label= "Collapsing boxes") )
    handles.append( mlines.Line2D([], [], marker='o',ls='', color="orangered", label= "Inconclusive algorithm") )
    plt.legend(handles=handles, fontsize = 10)
    plt.show()

### Let's go:

In [ ]:
W = W_BS09

test_wiring_triangle(
    W = W,
    Box1=non_local_boxes.utils.PR,  Box1_name = "$\mathbf{PR}$",
    Box2=non_local_boxes.utils.P_0, Box2_name = "$\mathbf{P}_0$",
    Box3=non_local_boxes.utils.P_1, Box3_name = "$\mathbf{P}_1$",
    box_grid_size = 60,  # 60
    max_box_power = 7,  # 10
    threshold = (3 + float(torch.sqrt(torch.tensor(6))))/6,
    Details_Are_Printed=True
)

--- 
# II. Test a wiring $\mathsf{W}$ in many Triangles

In [ ]:
def Test_Wiring(W_list, Fixed_box_in_drawings, Fixed_box_name, known_collapsing_W, Details_Are_Printed, compare_with_known_wirings=True, box_grid_size=60, max_box_power=20):
    # W_list is given as a list here

    if non_local_boxes.evaluate.nb_columns != 1:
        print("\n   WARNING: Please, set the number of columns to 1 (in `non_local_boxes.evaluate`).")
        return None

    W = torch.tensor(W_list)*1.
    if Details_Are_Printed: print("------ Number of known wirings: ", len(known_collapsing_W))
    print("\nW = ",W,"\n")
    if compare_with_known_wirings:
        for i in range(len(known_collapsing_W)):
            if torch.all(W == torch.tensor(known_collapsing_W[i])):
                print(" !!!  This wiring is already known: ",i+1,"th/ ",len(known_collapsing_W),"  !!!")
                return None
    
    threshold = (3 + float(torch.sqrt(torch.tensor(6))))/6 

    p = proportion_blue_points_in_a_segment(torch.t(W.repeat(1, 1)), PR, Fixed_box_in_drawings, 40, 40, threshold)
    texte = ""
    if Details_Are_Printed: texte ="------ "
    print(texte,"Proportion of blue points in the segment PR --", Fixed_box_name, ":   ", 100*p, "%")

    if p>0.4:
        boxes_to_be_tested = non_local_boxes.utils.boxes_to_be_tested
        for box_number in boxes_to_be_tested:
            mu, nu, sigma, tau = box_number
            if PL(mu,nu,sigma,tau).tolist() != Fixed_box_in_drawings.tolist():
                test_wiring_triangle(
                    W = W,
                    Box1=PR,  Box1_name = "PR",
                    Box2 = Fixed_box_in_drawings, Box2_name = Fixed_box_name,
                    Box3=PL(mu,nu,sigma,tau), Box3_name = f"""$\mathbf{{P}}_L^{{{mu}{nu}{sigma}{tau}}}$""",
                    box_grid_size = box_grid_size,
                    max_box_power = max_box_power,
                    threshold = threshold,
                    Details_Are_Printed = Details_Are_Printed
                )

    if Details_Are_Printed:  print("---------- FINISHED ----------")

### Let's go:

In [ ]:
known_collapsing_W = non_local_boxes.utils.known_collapsing_W

W = [0.,0.,1.,0.,1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,0.,0.,1.,0.,0.,1.,0.,1.,1.,0.,1.,0.,0.,1.,1.,0.,0.,1.]

non_local_boxes.utils.print_functions_from_wiring(W)
print("")

Test_Wiring(
    W,
    PL(1,0,0,0), "$\mathbf{P}_L^{0010}$",
    known_collapsing_W,
    Details_Are_Printed = True,
    compare_with_known_wirings=False,
    box_grid_size=60, max_box_power=10
)